In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!conda install -y -c rdkit rdkit;

In [ ]:
!pip install git+https://github.com/samoturk/mol2vec

In [ ]:
import itertools

from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit.Chem.Fragments as f
import rdkit.Chem.rdMolDescriptors as d
from rdkit.Chem import Lipinski

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import roc_curve,auc
# from sklearn.model_selection import StratifiedKFold
from scipy import interp
import matplotlib.pyplot as plt

In [ ]:
def load_data(path):
    return pd.read_csv(path, index_col=["INDEX"])


def get_smile(dataframe):
    data = dataframe.copy()
    data["rd_form"] = data.SMILES.apply(lambda x: Chem.MolFromSmiles(x))
    data.drop(["SMILES"], inplace=True, axis=1)
    return data


def get_feature(dataframe):
    data = dataframe.copy()
    
#     mol_model = word2vec.Word2Vec.load('../input/assignment4datas/model_300dim.pkl')

    data["num_atom"] = data["rd_form"].apply(lambda x:x.GetNumAtoms())
    data["mol_dr"] = data["rd_form"].apply(lambda x: d.CalcExactMolWt(x))
    data["halogen"] = data["rd_form"].apply(lambda x: f.fr_halogen(x))
    data["nhoh_count"] = data["rd_form"].apply(lambda x: Lipinski.NHOHCount(x))
    data["hacceptor_count"] = data["rd_form"].apply(lambda x: Lipinski.NumHAcceptors(x))
    data["hdonor_count"] = data["rd_form"].apply(lambda x: Lipinski.NumHDonors(x))
    data["hetero_count"] = data["rd_form"].apply(lambda x: Lipinski.NumHeteroatoms(x))
    data["rotatable_bond_count"] = data["rd_form"].apply(lambda x: Lipinski.NumRotatableBonds(x))
    data["aliphatic_ring_count"] = data["rd_form"].apply(lambda x: Lipinski.NumAliphaticRings(x))
    data["aromatic_ring_count"] = data["rd_form"].apply(lambda x: Lipinski.NumAromaticRings(x))
    

    # fingerprint
    fcfp_list = []    
    for fcpc in range(124):
        data["fcpc" + str(fcpc)] = 0
        fcfp_list.append("fcpc" + str(fcpc))
    fcpc_x = data["rd_form"].apply(lambda x: np.array(AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=124, useFeatures=True)))
    fcpc_vector_lists = list(itertools.chain(*fcpc_x))
    data.loc[:, fcfp_list] = np.array(fcpc_vector_lists).reshape(len(data),124)
    
    ecfp_list = []
    for ecpc in range(124):
        data["ecpc" + str(ecpc)] = 0
        ecfp_list.append("ecpc" + str(ecpc))
    ecpc_x = data["rd_form"].apply(lambda x: np.array(AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=124)))
    ecpc_vector_lists = list(itertools.chain(*ecpc_x))
    data.loc[:, ecfp_list] = np.array(ecpc_vector_lists).reshape(len(data),124)
    
#     # mol2vec
#     m2v_list = []
#     data['sentence'] = data['rd_form'].apply(lambda x: mol2alt_sentence(x, radius=2))  
#     for m2v_idx in range(300):
#         data["m2v" + str(m2v_idx)] = 0
#         m2v_list.append("m2v" + str(m2v_idx))
#     m2v = [DfVec(x) for x in sentences2vec(data['sentence'], mol_model, unseen='UNK')]
#     m2v = np.array([x.vec for x in m2v])
#     data.loc[:,m2v_list] = m2v.reshape(len(data),300)
#     data.drop(["sentence"], inplace=True, axis=1)
    
    return data


def cal_auc(prob, labels):
    f = list(zip(prob, labels))
    rank = [values2 for values1, values2 in sorted(f, key=lambda x: x[0])]
    rankList = [i + 1 for i in range(len(rank)) if rank[i] == 1]
    posNum = 0
    negNum = 0
    for i in range(len(labels)):
        if (labels[i] == 1):
            posNum += 1
        else:
            negNum += 1
    auc = (sum(rankList) - (posNum * (posNum + 1)) / 2) / (posNum * negNum)
    return auc


In [ ]:
def binary_bayes(x_train, x_val,  y_train, y_val):
    model = BernoulliNB()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction


def guassian_bayes(x_train, x_val,  y_train, y_val):
    model = GaussianNB()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction


def multi_bayes(x_train, x_val,  y_train, y_val):
    model = MultinomialNB()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction


def decision_tree(x_train, x_val,  y_train, y_val):
    model = DecisionTreeClassifier()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction


def mlp(x_train, x_val,  y_train, y_val):
    model = MLPClassifier()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction


# def random_forest(x_train, x_val,  y_train, y_val):
#     model = RandomForestClassifier()
#     model.fit(x_train, y_train)
#     prediction = model.predict_proba(x_val)
#     auc = cal_auc(prediction[:, 1], np.array(y_val))
#     return model, auc, prediction
def random_forest(x_train, x_val,  y_train, y_val, params=None):
    if params == None:
        model = RandomForestClassifier()
    else:
        model = RandomForestClassifier(**params)
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction

def light_boost(x_train, x_val,  y_train, y_val):
    model = lgb.LGBMClassifier()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction


def extreme_boost(x_train, x_val,  y_train, y_val):
    model = xgb.XGBClassifier()
    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    auc = cal_auc(prediction[:, 1], np.array(y_val))
    return model, auc, prediction

In [ ]:
# load data different features 
train_atom = pd.read_csv('/kaggle/input/smile-feature/atom.csv', index_col=['INDEX'])
train_m2v = pd.read_csv('/kaggle/input/smile-feature/m2v.csv', index_col=['INDEX'])
train_fcfp = pd.read_csv('/kaggle/input/smile-feature/fcpc.csv', index_col=['INDEX'])
train_ecfp = pd.read_csv('/kaggle/input/smile-feature/ecpf.csv', index_col=['INDEX'])
y = train_atom['ACTIVE']
drop_list = ['ACTIVE', 'rd_form', 'COO', 'OH', 'Nhpyrrole', 'SH', 'ketones'] # this features' means equal 0
train_atom = train_atom.drop(drop_list, axis=1)
train_m2v = train_m2v.drop(['rd_form', 'ACTIVE'], axis=1)
train_fcfp = train_fcfp.drop(['rd_form', 'ACTIVE'], axis=1)
train_ecfp = train_ecfp.drop(['rd_form', 'ACTIVE'], axis=1)

In [ ]:
train_atom_fcfp = pd.concat([train_atom, train_fcfp], axis=1)
train_atom_ecfp = pd.concat([train_atom, train_ecfp], axis=1)
train_atom_m2v = pd.concat([train_atom, train_m2v], axis=1)
# train_fcfp_ecfp = pd.concat([train_fcfp, train_ecfp], axis=1)
train_atom_fcfp_ecfp = pd.concat([train_atom, train_fcfp, train_ecfp], axis=1)
train_all = pd.concat([train_atom, train_fcfp, train_ecfp, train_m2v], axis=1)

In [ ]:
# split and normalization

def split(data, y):
    x_train, x_val, y_train, y_val = train_test_split(data, y, test_size=0.2, random_state=1)
    return x_train, x_val, y_train, y_val


def trans(x_train, x_val, label_list):
    train = x_train.loc[:, label_list]
    val = x_val.loc[:, label_list]
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    scaler = MinMaxScaler()
    kbd = KBinsDiscretizer(n_bins=10, encode="ordinal")
    train = imp_mean.fit_transform(train)
    train = scaler.fit_transform(train)
    train = kbd.fit_transform(train)
    val = imp_mean.transform(val)
    val = scaler.transform(val)
    val = kbd.transform(val)
    x_train.loc[:, label_list] = np.array(train)
    x_val.loc[:, label_list] = np.array(val)
    return x_train, x_val

# split and normalization

# norm features
norm_features = list(train_atom.columns)
# basic models results, without tuning 
def check(train, y, norm=False, norm_list=norm_features):
    x_train, x_val, y_train, y_val = split(train, y)
    if norm == True:
        x_train, x_val = trans(x_train, x_val, norm_list)
    lb, lb_auc, lb_prediction = light_boost(x_train,x_val,y_train, y_val)
    eb, eb_auc, en_prediction = extreme_boost(x_train,x_val,y_train, y_val)
    rf, rf_auc, rf_prediction = random_forest(x_train,x_val,y_train, y_val)
    baseline_r = [rf_auc, lb_auc, eb_auc]
    print("rf auc without tuning: " + str(rf_auc))
    print("lgbm auc without tuning: " + str(lb_auc))
    print("xgboost auc without tuning: " + str(eb_auc))
    return baseline_r

In [ ]:
# baseline
baseline_rows = ["basic", "fcfp", "ecfp", "m2v", "basic+fcfp", "basic+ecfp",
                "basic+m2v", "basic+fcfp+ecfp", "all"]
baseline_columns = ["rf", "lgbm", "xgbm"]
r1 = check(train_atom, y, norm=True)
r2 = check(train_fcfp, y)
r3 = check(train_ecfp, y)
r4 = check(train_m2v, y, norm_list = list(train_m2v.columns))
r5 = check(train_atom_fcfp, y, norm=True)
r6 = check(train_atom_ecfp, y, norm=True)
r7 = check(train_atom_m2v, y, norm=True, norm_list = list(train_atom.columns)+
      list(train_m2v.columns))
r8 = check(train_atom_fcfp_ecfp, y, norm=True)
r9 = check(train_all, y, norm=True, norm_list = list(train_atom.columns)+
      list(train_m2v.columns))
baseline_r = [r1, r2, r3, r4, r5, r6, r7, r8, r9]
r_df = pd.DataFrame(baseline_r, columns=baseline_columns, index=baseline_rows)
r_df.to_csv('baseline_results.csv')

In [ ]:
def hyper_tuned_rf(x_train, x_val,  y_train, y_val, model, kf, params):
    grid_search = GridSearchCV(model, param_grid=params, cv=kf, scoring='roc_auc')
    grid_search.fit(x_train, y_train)
    best_params = grid_search.best_params_
    update_model = RandomForestClassifier(**best_params)
    update_model.fit(x_train, y_train)
    update_prediction = update_model.predict_proba(x_val)
    update_auc = cal_auc(update_prediction[:, 1], np.array(y_val))
    
    return update_model, update_prediction, update_auc, best_params, grid_search

In [ ]:
# plot the curve and save the results

def rslt_plot(x_train, y_train, params, round_):
    tprs=[]
    aucs=[]
    mean_fpr=np.linspace(0,1,100)
    i = 0
    kf = KFold(n_splits=5, random_state=42, shuffle=False)
    fig_name = f'rf_result_fig{round_}'
    fig = plt.figure()
    for train_index,test_index in kf.split(x_train):
        # divide dataset into validation
        X_train,X_test = x_train.iloc[train_index,:],x_train.iloc[test_index,:]
        Y_train,Y_test = y_train.iloc[train_index],y_train.iloc[test_index]
        rf = RandomForestClassifier(**params)
        rf.fit(X_train, Y_train)
        Y_pred = rf.predict_proba(X_test)
#         print(np.array(Y_pred[:,1]), Y_test)
        fpr,tpr,thresholds=roc_curve(np.array(Y_test),Y_pred[:,1])

        tprs.append(interp(mean_fpr,fpr,tpr))
        tprs[-1][0]=0.0
        # calculate auc
        roc_auc=auc(fpr,tpr)
        aucs.append(roc_auc)
        # need plt.plot(fpr,tpr)
        plt.plot(fpr,tpr,lw=1,alpha=0.3,label='ROC fold %d(area=%0.2f)'% (i,roc_auc))
        i +=1    
    plt.plot([0,1],[0,1],linestyle='--',lw=2,color='tomato',label='Active',alpha=.8)
    mean_tpr=np.mean(tprs,axis=0)
    mean_tpr[-1]=1.0
    # the mean auc
    mean_auc=auc(mean_fpr,mean_tpr)
    std_auc=np.std(tprs,axis=0)
    plt.plot(mean_fpr,mean_tpr,color='teal',label=r'Mean ROC (area=%0.2f)'%mean_auc,lw=2,alpha=.8)
    std_tpr=np.std(tprs,axis=0)
    tprs_upper=np.minimum(mean_tpr+std_tpr,1)
    tprs_lower=np.maximum(mean_tpr-std_tpr,0)
    plt.fill_between(mean_tpr,tprs_lower,tprs_upper,color='cornflowerblue',alpha=.2)
    plt.xlim([-0.05,1.05])
    plt.ylim([-0.05,1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC')
    plt.legend(loc='lower right')
    fig.savefig(f'{fig_name}.png')

In [ ]:
rf_params = {
    'n_estimators': [225, 300, 400],
    'class_weight': ['balanced_subsample'],
    'oob_score': [True],
    'criterion': ['entropy'],
    'max_depth': [30, 40]
}
kf = KFold(n_splits=5, random_state=42, shuffle=False)
x_train, x_val, y_train, y_val = split(train_atom_fcfp_ecfp, y)
x_train, x_val = trans(x_train, x_val, norm_features)
rf, auc, pred = random_forest(x_train, x_val,  y_train, y_val)
update_rf, update_rf_prediction, update_rf_auc, best_params_rf, grid_search_rf = hyper_tuned_rf(x_train, x_val, y_train, y_val,
                                                                               model=rf, kf=kf, params=rf_params)


In [ ]:
print("rf AUC without tuning:" + str(auc))
print("best params of RF: " + str(best_params_rf))
print("rf AUC with best params:" + str(update_rf_auc))

In [ ]:
def save_results(results, model_name):
    df = pd.DataFrame.from_dict(grid_search_rf.cv_results_)
    path = f'output{model_name}.csv'
    df.to_csv(path)
    print("saved " + model_name + ' as ' + path)

In [ ]:
save_results(grid_search_rf, "rf_tuning")

In [ ]:
grid_search_rf.cv_results_

In [ ]:
rf_final_params = {
    'n_estimators': 400,
    'class_weight': 'balanced_subsample',
    'oob_score': True,
    'criterion': 'entropy',
    'max_depth': 30
}
norm_list = list(train_atom.columns)
# x_train, x_val, y_train, y_val = split(train_atom[:1000], y[:1000])
x_train, x_val, y_train, y_val = split(train_atom_fcfp_ecfp, y)
x_train, x_val = trans(x_train, x_val, norm_list)

# plot
# rslt_plot(x_train, y_train, rf_final_params, "400+30")

In [ ]:
x_train

In [ ]:
# output the results on test dataset
train_data = train_atom_fcfp_ecfp
test_data = pd.read_csv('../input/assignment4datas/test_smiles.csv', index_col=['INDEX'])

# add features
test_data = get_feature(get_smile(test_data))
# store
test_data.to_csv('test_features.csv')



In [ ]:
# normalization and remove some columns
drop_list = ['rd_form'] # this features' means equal 0
test_data = test_data.drop(drop_list, axis=1)
# norm_list = list(train_atom.columns)
x_train, x_val, y_train, y_val = split(train_data, y)
x_train_ = x_train
x_train, x_val = trans(x_train, x_val, norm_features)
x_train_, test_data = trans(x_train_, test_data, norm_features) 

In [ ]:
final_rf, final_auc, final_val_prediction = random_forest(x_train, x_val,  y_train, y_val, rf_final_params)
print("final auc on val data: " + str(final_auc))
final_results = final_rf.predict_proba(test_data)

In [ ]:
rr = pd.DataFrame(final_results, columns=['active=0.0', 'active=1.0'], index=list(test_data.index))
rr.to_csv('final_results.csv')

In [ ]:
cnt = 0
for i in range(len(rr)):
    if rr.loc[i+121375, 'active=0.0']>0.5:
        cnt += 1

cnt